In [1]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("people.json") as f:
    file_data = json.load(f)
db.users.insert_many(file_data)


In [2]:
for x in db.users.aggregate(
    [
        {
            "$project": {
                "_id": 0,
                "birthday": {"$convert": {"input": "$dob.date", "to": "date"}},
            }
        },
        {
            "$group": {
                "_id": {"birthYear": {"$isoWeekYear": "$birthday"}},
                "num_people": {"$sum": 1},
            },
        },
        {"$sort": {"num_people": -1}},
        {"$limit": 5},
    ]
):
    pprint(x)


{'_id': {'birthYear': 1955}, 'num_people': 113}
{'_id': {'birthYear': 1961}, 'num_people': 111}
{'_id': {'birthYear': 1993}, 'num_people': 110}
{'_id': {'birthYear': 1960}, 'num_people': 110}
{'_id': {'birthYear': 1975}, 'num_people': 107}


In [3]:
client.close()
